In [1]:
import enchant
import re
import nltk
from nltk.collocations import *
from nltk.tokenize import sent_tokenize
import string
import datrie
from pymystem3 import Mystem
m = Mystem()

In [2]:
d = enchant.Dict("ru_RU")

In [3]:
r_alphabet = re.compile(u'[0-9а-яА-ЯёЁ]+|[.,:;?!]+')

In [4]:
d1=datrie.Trie.load('/media/sf_communalflat/ihatelinguistics/hyphen_ext.trie')
d2=datrie.Trie.load('/media/sf_communalflat/ihatelinguistics/wrong_hyphen.trie')

def lookup(tokens):
    s = []
    for i in range(len(tokens)):
        s.append(tokens[i])
        if len(s)>1:
            if s[-2].lower()+' '+s[-1].lower() in d1.keys():
                word=d1[s[-2].lower()+' '+s[-1].lower()]
                s = s[:-2]
                s.append(word)
    return s     

def lookup_non_hyphens(tokens):
    s=[]
    for i in tokens:
        if '-' in i and i in d2.keys():
            for w in d2[i].split():
                s.append(w)
        else:
            s.append(i)
    return s

def assemble_string(tokens):
    s=''
    for i in range(len(tokens)):
        if i==0:
            word=tokens[i][0].upper()+tokens[i][1:]
            s = s+word
        if i>0:
            if tokens[i][0] in string.punctuation:
                s = s+tokens[i]
            else:
                s = s + ' ' + tokens[i]
    return s

In [5]:
def get_prob(s):
    prob=0.0
    if s in probdict.keys():
        prob+=probdict[s]
    return prob+0.0001

In [6]:
def similarity(s1, s2):
    v=0
    for i in s1:
        if i in s2:
            s2 = s2.replace(i, '')
            v+=1
    return v

In [7]:
probdict = datrie.Trie.load('lmodel.trie')

def correct_text(sentences):
    corrected_text=[]
    for i in sentences:
        tokens = r_alphabet.findall(i.lower())
        tokens1=lookup(tokens)
        tokens2=lookup_non_hyphens(tokens1)
        sent=[]
        for word in tokens2:
            if d.check(word)==True:
                sent.append(word)
            elif d.check(word)==False and not d.suggest(word):
                sent.append(word)
            elif d.check(word)==False and d.suggest(word):
                possible=[]
                for i in d.suggest(word):
                    possible.append(i)
                if len(possible)==1:
                    sent.append(possible[0])
                elif len(possible)>1:
                    probs={}
                    lemmas={}
                    reverse_lemmas={}
                    windows=[]
                    for i in possible:
                        lemmas[i]=m.lemmatize(i)[0]
                        reverse_lemmas[m.lemmatize(i)[0]]=i
                        if len(tokens2)==1:
                            probs[i]=1-(levenshtein(i, word))
                        elif len(tokens2)>=2 and tokens2[0]==word:
                            windows.append('pad'+' '+lemmas[i]+' '+m.lemmatize(tokens2[1])[0])
                        elif len(tokens2)>=2 and tokens2[-1]==word:
                            windows.append(m.lemmatize(tokens2[-2])[0]+' '+lemmas[i]+' '+'pad')
                        else:
                            windows.append(m.lemmatize(tokens2[tokens2.index(word)-1])[0]+' ' +lemmas[i]+' '+m.lemmatize(tokens2[tokens2.index(word)+1])[0])
                    for i in windows:
                        window_tokens=i.split()
                        probs[reverse_lemmas[window_tokens[1]]]=get_prob('[] []'.format(window_tokens[0],window_tokens[1]))*get_prob('[] []'.format(window_tokens[1],window_tokens[2]))
                    max_prob=max(probs, key=probs.get)
                    extra_probs=[]
                    for i in probs.items():
                        if i[1]==probs[max_prob]:
                            extra_probs.append(i[0])
                    if len(extra_probs)>1:
                        extra_dict={}
                        for i in extra_probs:
                            extra_dict[i]=similarity(i, word)
                        val_l=max(extra_dict, key=extra_dict.get)
                        sent.append(val_l)
                    else:
                        sent.append(extra_probs[0])
        corrected_text.append(assemble_string(sent))
    return ' '.join(i for i in corrected_text)

In [12]:
test=open('/media/sf_communalflat/ihatelinguistics/spell_test.txt').readlines()

In [13]:
test_lines=[]
i=0
while i<=len(test):
    test_lines.append(test[i])
    i+=3
with open('spelltest_source.txt', 'w', encoding='utf-8') as f:
    f.writelines(test_lines)

In [15]:
corr_lines=[]
i=1
while i<=len(test):
    corr_lines.append(test[i])
    i+=3
with open('spelltest_answer.txt', 'w', encoding='utf-8') as f:
    f.writelines(corr_lines)

In [16]:
checked_lines=[]
for i in test_lines:
    checked_lines.append(correct_text([i]))

In [19]:
with open('spelltest_checked1.txt', 'w', encoding='utf-8') as f:
    for i in checked_lines:
        f.write(i+'\n')